<a href="https://colab.research.google.com/github/anhky/Full_YOLO/blob/master/YOLO_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Giới thiệu chung**

## 1.1. YOLO là gì 
You only look once (Chỉ cẩn nhìn 1 lần) là có thể phát hiện ra vật thể (Rất nhanh rất nhanh)

YOLO không phải là thuật toán tốt nhất nhưng là thuật toán nhanh nhất trong các mô hình object detection => Real time

**Mục tiêu**
- Dự báo nhãn dán (lable) bài toán phân lớp classification 
- Xác định location của vật thể ( xác định đươc nhiều vật thể trong 1 ảnh)

**Yêu cầu**
- Convolution Neural Network 
- Bounding box, anchor box, Feature map, Non-max suppression


# **2. Kiến trúc mạng YOLO**
Bao gồm 2 thành phần
- base network : Là các mạng convolution làm nhiệm vụ trích xuất đặc trưng. 
- Extra Layers: Được áp dụng để phát hiện vật thể trên feature map của base network.

  ![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h1.png)**Hình 1:** Sơ đồ kiến trúc mạng YOLO.
- base network: DarkNet Architecture.Output của base network là mộ feature map 7x7x1024, feature map 7x7x30
- Etra layers: các phần phía sau

YOLO verion 3: Sử dụng mạng feature extractor là darknet-53
Mạng này bao gồm
-  53 convolutional layers kết nối liên tiếp. Mỗi layer được theo sau bởi 1 batch normalization + activation Leaky Relu + sử dụng bộ lọc (filter = 2) (để giảm thiểu số lượng tham số)

  ![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h2.png)Hình 2: Các layer trong mạng darknet-53




# **3. Input và Output của YOLO**



## 3.1 Input 
Các bức ảnh 

Các bức ảnh sẽ được scale chung về một kích thước phù hợp với input shape trước khi vào các convolution và sẽ được giảm dần theo cấp số nhân 2 khi đi qua các layer convolution, ta thu được feature map kích thước tương đối nhỏ để dự báo trên từng ô của feature 

**input shape của YOLO**
- input 416x416 => feature map 13x13, 26x26 và 52x52
- input 608x608 => feature map 19x19, 38x38 và 72x72

## 3.2 Output

Output của mô hình YOLO là 1 vector bao gồm các thành phần

  ![Texte alternatif…](https://latex.codecogs.com/png.latex?\inline&space;\dpi{150}&space;\bg_green&space;y^{T}&space;=&space;[p_0,\langle\underbrace{t_x,&space;t_y,&space;t_w,&space;t_h}_{\text{bounding&space;box}}\rangle,&space;\langle\underbrace{p_1,&space;p_2,...,&space;p_c}_{\text{scores&space;of&space;c&space;classes}}\rangle])

Trong đó 

- ![Texte alternatif…](https://latex.codecogs.com/png.latex?\inline&space;\dpi{150}&space;\bg_green&space;p_0)  là xác suất dự báo vật thể xuất hiện trong bounding box.

- ![Texte alternatif…](https://latex.codecogs.com/png.latex?\inline&space;\dpi{150}&space;\bg_green&space;\langle\underbrace{t_x,&space;t_y,&space;t_w,&space;t_h}_{\text{bounding&space;box}}\rangle) giúp xác định bounding box

- ![Texte alternatif…](https://latex.codecogs.com/png.latex?\inline&space;\dpi{150}&space;\bg_green&space;\langle\underbrace{p_1,&space;p_2,...,&space;p_c}_{\text{scores&space;of&space;c&space;classes}}\rangle) là vector phân phối xác xuất dự báo


Việc hiểu output khá là quan trọng để chúng ta cấu hình tham số chuẩn xác khi huấn luyện model qua các open source như darknet. 

Như vậy output sẽ được xác định theo số lượng classes theo công thức (n_class+5).

Nếu huấn luyện 80 classes thì bạn sẽ có output là 85. 

Trường hợp bạn áp dụng 3 anchors/cell thì số lượng tham số output sẽ là: (n_class+5)×3=85×3=255

![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h3.png) **HÌNH 3** Kiến trúc ouput của YOLO 

- feature map kích thước 13x13. 
- Trên mỗi cell lấy ra 3 anchor boxes: Tâm của anchor Box1, Box2, Box3 trùng với cell 
- Khi đó output của YOLO là một vector concatenate của 3 bounding boxes










# **4. Dự báo trên nhiều feature**
Feature map ban đầu có kích thước nhỏ giúp => dự đoán các object kích thước lớn.

Feature map sau có kích thước lớn, mà anchor box không thay đổi => dự đoán các object kích thước nhỏ

  ![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h4.png) **Hình 4** Các feature maps của mạng YOLOv3 với input shape là 416x416, output là 3 feature maps có kích thước lần lượt là 13x13, 26x26 và 52x52.

Trên mỗi một cell của các feature map chúng ta sẽ áp dụng 3 anchor box để dự đoán vật thể. Như vậy số lượng các anchor box khác nhau trong một mô hình YOLO sẽ là 9 (3 featue map x 3 anchor box).

Đồng thời trên một feature map hình vuông S x S, mô hình YOLOv3 sinh ra một số lượng anchor box là: S x S x 3. Như vậy số lượng anchor boxes trên một bức ảnh sẽ là: (13×13+26×26+52×52)×3=10647(anchor boxes)

Đây là một số lượng rất lớn và là nguyên nhân khiến quá trình huấn luyện mô hình YOLO vô cùng chậm bởi chúng ta cần dự báo đồng thời nhãn và bounding box trên đồng thời 10647 bounding boxes.

**Một số lưu ý khi huấn luyện YOLO:**

Khi huấn luyện YOLO sẽ cần phải có RAM dung lượng lớn hơn để save được 10647 bounding boxes như trong kiến trúc này.

Không thể thiết lập các batch_size quá lớn như trong các mô hình classification vì rất dễ Out of memory.

Package darknet của YOLO đã chia nhỏ một batch thành các subdivisions cho vừa với RAM.

Thời gian xử lý của một step trên YOLO lâu hơn rất rất nhiều lần so với các mô hình classification. Do đó nên thiết lập steps giới hạn huấn luyện cho YOLO nhỏ.

Đối với các tác vụ nhận diện dưới 5 classes, dưới 5000 steps là có thể thu được nghiệm tạm chấp nhận được. Các mô hình có nhiều classes hơn có thể tăng số lượng steps theo cấp số nhân tùy bạn. ( sẽ thực hiện bài tập ở bài sau)

# **5. Anchor box**
Anchor box làm cơ sở ước lượng để tìm được bounding box trong vậy thể.

Thuật toán regression box sẽ tinh chỉnh anchor box để tạo ra bounding box

- Một vật thể trong hình ảnh huấn luyện được phân bổ về một cell trên featute map có chứa điểm mid poin của vật thể.
- Từ trong cell ta xác định các anchor bao quanh vật thể, nếu có nhiều hơn 2 anchor box sẽ chọn anchor box có IoU so với ground truth bounding box là cao nhất .

***=> Xác định vật thể cần 2 thành phần ( cell và anchor box)***

![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h5.png)**Hình 5:** Xác định anchor box cho một vật thể

Một số trường hợp 2 vật thể bị trùng mid point, mặc dù rất hiếm khi xảy ra, thuật toán sẽ rất khó xác định được class cho chúng.

![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h6.jpg) **Hinh 6:** Hai vật thể trùng mid point và cùng thuộc 1 cell => Thuật toán cần thêm lượt tiebreak để quyết định đâu là class cho cell


# **6. Hàm loss function** 

SSD và YOLO 

Hàm loss function chia thành 2 phần: localization loss và confidence loss.
- location loss: Hàm mất mát của bounding box dự báo so với thực tế.
- confidence los: Hàm phân phối xác suất. 

Mình chưa hiểu rõ : Ai đó có thể góp ý với mình qua gmail: anhkypropy@gmail.com





# **7. Dự báo bounding box**
(YOLO version 2 và version 3) 
Tọa độ của một bounding box sẽ được xác định dựa trên đồng thời cả anchor box và cell mà nó thuộc về = >  giúp kiểm soát vị trí của bounding box dự đoán đâu đó quanh vị trí của cell và bounding box mà không vượt quá xa ra bên ngoài giới hạn version 1

![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h7.png)

**Hình 7** Công thức ước lượng bounding box từ anchor box.

# **8. Non-max suppression**
Xem lại mục Non-max suppression để hiểu rõ hơn.

Do thuật toán YOLO dự báo ra rất nhiều bounding box trên một bức ảnh nên đối với những cell có vị trí gần nhau, khả năng các khung hình bị overlap là rất cao => Dùng non-max suppression để giảm bớt số lượng khung hình có độ chính xác thấp

![Texte alternatif…](https://phamdinhkhanh.github.io/assets/images/20200309_YOLOAlgorithm/h8.png) 

**Hình 8** Sử dụng non-max-suppression 

**Các bước của non-max suppression:**
- Step 1: giảm số lượn bounding box bằng cách lọc bỏ nhũng bounding box có xác suất chứa vật thể nhỏ hơn một ngưỡng threhold, thường là 0.5
- Step 2: Đối với nhũng bounding box giao nhau, non-max suppression chọn ra những bounding box có xác xuất chứa vật thể là lớn nhất. Sau đó tính chỉ số giao thoa IoU với các bounding box còn lại.

Nếu chỉ số lớn hơn ngưỡng threhold cho phép chứng tỏ 2 bounding box bị overlap rất cao (Qua mặt bước 1) => (Đến bước 2) xóa bỏ các bounding box có xác xuất chưa vật thể thấp => Thu dược bounding box duy nhất cho vật thể. 

# **9.Cũng cố**
Cũng cố lại kiến thức về anchor box, bounding box, feature nap, Non-max suppression, Đặc biệt là nguyên lý hoạt động của Convolution Neural Network.